In [1]:
import os
import copy
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlp
%matplotlib inline
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM, BartForConditionalGeneration,BartTokenizer
import shap
import scipy as sp
import nlp
import torch
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option("max_rows", None)
from shap.utils import assert_import, record_import_error, safe_isinstance, make_masks, OpChain, MaskedModel
from shap import maskers, links
from transformers.file_utils import ModelOutput
#from shap.utils import cal_conditional_logits
import sys

In [2]:
def cuda_memory_stats():
    t = torch.cuda.get_device_properties(0).total_memory
    c = torch.cuda.memory_cached(0)
    a = torch.cuda.memory_allocated(0)
    f = c-a
    print(t)
    print(c)
    print(a)
    print(f)

In [3]:
cuda_memory_stats()

4294967296
0
0
0


torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved


In [56]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-medium").cuda()

In [54]:
s="Today the weather is really nice and I am planning on "

inputs = tokenizer([s + tokenizer.eos_token], max_length=512, return_tensors='pt')
input_ids=inputs['input_ids'].cuda()

summary_ids = model.generate(input_ids)

summary=[tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]

print(summary)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


['Today the weather is really nice and I am planning on The first of two posts on the']


In [31]:
summary_ids

tensor([[13921,  1637,  2822, 12157,    30, 50256, 26788, 24779, 12157,   837,
           475,   340,   635, 24779,   257,  1256,   286,  1243,   326,   787,
           345,  3772,   764, 50256]], device='cuda:0')

In [53]:
input_ids

tensor([[ 8421, 25523,   534, 10701,   284,  8706,    11,  3505,   284,  2353,
           777,  3709,   220]], device='cuda:0')

In [28]:
tokenizer.eos_token

'<|endoftext|>'

In [ ]:
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt').cuda()

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:Do you want black or white coffee ? 
DialoGPT: I'm not sure what you mean by that.
>> User:tennis or football?
DialoGPT: I'm not sure what you mean by that.
>> User:how are you?
DialoGPT: I'm good.
